In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

In [2]:
filename = "story.txt"
f = open(filename, 'r')

raw_text = f.read()

In [3]:
raw_text[0:1000]

'Once upon a time, there was a boy named Mike\nHe was raised in the hood, life was never quite\nGrowing up, he faced many struggles and strife\nBut through it all, he had one love in his life\n\nRap music was his escape from reality\nThe beats and lyrics helped him to see\nA world beyond his troubled neighborhood\nHe knew that with hard work, he could change his mood\n\nHe started writing rhymes and practicing his flow\nIn the mirror, he would rap and watch himself grow\nHe knew that if he could just make it to the top\nHe could change his life, and make a better stop\n\nOne day, he had the chance to perform on stage\nIn front of a crowd, he killed it, he killed it with rage\nHe had finally made it, he had achieved his dream\nAnd now he is a successful rapper, or so it seems\n\nHe never forgot where he came from, and he never will\nHe always remember the struggles that he had to fulfill\nHe is now a role model to kids in the hood\nShowing them that with hard work, they too could\n\nRis

In [4]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]', r'', processed_text)

processed_text

'once upon a time, there was a boy named mike\nhe was raised in the hood, life was never quite\ngrowing up, he faced many struggles and strife\nbut through it all, he had one love in his life\n\nrap music was his escape from reality\nthe beats and lyrics helped him to see\na world beyond his troubled neighborhood\nhe knew that with hard work, he could change his mood\n\nhe started writing rhymes and practicing his flow\nin the mirror, he would rap and watch himself grow\nhe knew that if he could just make it to the top\nhe could change his life, and make a better stop\n\none day, he had the chance to perform on stage\nin front of a crowd, he killed it, he killed it with rage\nhe had finally made it, he had achieved his dream\nand now he is a successful rapper, or so it seems\n\nhe never forgot where he came from, and he never will\nhe always remember the struggles that he had to fulfill\nhe is now a role model to kids in the hood\nshowing them that with hard work, they too could\n\nris

In [5]:
print("Corpus Length: {}".format(len(processed_text)))

characters = sorted(list(set(processed_text)))
print("Total Characters: {}".format(len(characters)))

character_indices = dict((c, i) for i,c in enumerate(characters))
indices_characters = dict((i, c) for i, c in enumerate(characters))

Corpus Length: 1073
Total Characters: 27


In [6]:
character_indices

{'\n': 0,
 ' ': 1,
 ',': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'y': 26}

In [7]:
indices_characters

{0: '\n',
 1: ' ',
 2: ',',
 3: 'a',
 4: 'b',
 5: 'c',
 6: 'd',
 7: 'e',
 8: 'f',
 9: 'g',
 10: 'h',
 11: 'i',
 12: 'j',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'q',
 20: 'r',
 21: 's',
 22: 't',
 23: 'u',
 24: 'v',
 25: 'w',
 26: 'y'}

In [8]:
maxlen = 40
step = 5

sentences = []
next_characters = []

for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i+maxlen])
    next_characters.append(processed_text[i + maxlen])

In [9]:
sentences

['once upon a time, there was a boy named ',
 'upon a time, there was a boy named mike\n',
 'a time, there was a boy named mike\nhe wa',
 'e, there was a boy named mike\nhe was rai',
 'ere was a boy named mike\nhe was raised i',
 'as a boy named mike\nhe was raised in the',
 'boy named mike\nhe was raised in the hood',
 'amed mike\nhe was raised in the hood, lif',
 'mike\nhe was raised in the hood, life was',
 'he was raised in the hood, life was neve',
 's raised in the hood, life was never qui',
 'sed in the hood, life was never quite\ngr',
 'n the hood, life was never quite\ngrowing',
 ' hood, life was never quite\ngrowing up, ',
 ', life was never quite\ngrowing up, he fa',
 'e was never quite\ngrowing up, he faced m',
 ' never quite\ngrowing up, he faced many s',
 'r quite\ngrowing up, he faced many strugg',
 'te\ngrowing up, he faced many struggles a',
 'owing up, he faced many struggles and st',
 ' up, he faced many struggles and strife\n',
 'he faced many struggles and strife\n

In [10]:
next_characters

['m',
 'h',
 's',
 's',
 'n',
 ' ',
 ',',
 'e',
 ' ',
 'r',
 't',
 'o',
 ' ',
 'h',
 'c',
 'a',
 't',
 'l',
 'n',
 'r',
 'b',
 'h',
 'h',
 'a',
 'h',
 'd',
 ' ',
 ' ',
 'i',
 'f',
 'a',
 's',
 'a',
 's',
 'a',
 'r',
 'e',
 'y',
 ' ',
 's',
 ' ',
 'c',
 'l',
 'h',
 'o',
 '\n',
 'r',
 'e',
 ' ',
 't',
 'l',
 'e',
 'o',
 'd',
 'k',
 't',
 'w',
 'h',
 'w',
 ' ',
 'o',
 'c',
 'e',
 ' ',
 '\n',
 's',
 'e',
 'i',
 ' ',
 'e',
 'd',
 'c',
 'n',
 's',
 'w',
 't',
 'i',
 ',',
 'w',
 ' ',
 'a',
 'a',
 'h',
 'l',
 'o',
 ' ',
 ' ',
 ' ',
 'e',
 'l',
 's',
 'k',
 ' ',
 'h',
 'p',
 'c',
 ' ',
 'g',
 's',
 'e',
 'd',
 'e',
 'e',
 ' ',
 '\n',
 ' ',
 ' ',
 'a',
 'e',
 'n',
 'o',
 'f',
 'o',
 'a',
 'n',
 'n',
 ' ',
 'o',
 'h',
 'l',
 'i',
 'e',
 'l',
 't',
 'h',
 'e',
 'h',
 'i',
 'y',
 'e',
 ' ',
 'a',
 'h',
 'd',
 ' ',
 'm',
 ' ',
 'h',
 ' ',
 'c',
 'f',
 'a',
 ',',
 's',
 ' ',
 's',
 ' ',
 'r',
 'g',
 'h',
 'h',
 'm',
 'o',
 'n',
 ' ',
 'r',
 'l',
 'a',
 's',
 'e',
 ' ',
 's',
 'g',
 't',
 'h',
 'd',
 

In [11]:
print("Vectorization")
device = 'cuda'
x = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.float64)
y = np.zeros((len(sentences), len(characters)), dtype=np.float64)

for i, sentence in enumerate(sentences):
    for t, character, in enumerate(sentence):
        x[i, t, character_indices[character]] = 1
    y[i, character_indices[next_characters[i]]] = 1
    
x = torch.tensor(x).float().to(device)
y = torch.tensor(y).float().to(device)

Vectorization


In [12]:
x.shape

torch.Size([207, 40, 27])

In [13]:
x = torch.flatten(x, start_dim=1)

x.shape

torch.Size([207, 1080])

In [14]:
y.shape

torch.Size([207, 27])

In [15]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [16]:
def callback(model):
    start = 0
    stop = len(processed_text) - maxlen - 1

    #print("Start: {}".format(start))
    #print("Stop: {}".format(stop))

    start_index = random.randint(start, stop)

    #print("Start Index: {}".format(start_index))

    sentence = processed_text[start_index: start_index + maxlen]

    #print("Sentence: {}".format(sentence))
    #print("Sentence Length: {}".format(len(sentence)))

    generated = ''

    for i in range(400):
        x_predictions = np.zeros((1, maxlen, len(characters)))

        for t, char in enumerate(sentence):
            x_predictions[0, t, character_indices[char]] = 1

            # print(x_predictions)
        x_predictions = torch.tensor(x_predictions).float().to(device)
        x = torch.flatten(x_predictions, start_dim=1)

        preds = model.forward(x)[0].detach().cpu().numpy()

        next_index = sample(preds)
        #print("next_index: {}".format(next_index))
        next_char = indices_characters[next_index]
        #print("next_char: {}".format(next_char))

        generated += next_char
        sentence = sentence[1:] + next_char

    return sentence

In [17]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.hidden = nn.Linear(input_dim, 500)
        self.output = nn.Linear(500, output_dim)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # f(x) = a(f(x))
        x = self.relu(self.hidden(x))
        y = self.sigmoid(self.output(x))

        return y

In [18]:
model = MultiLayerPerceptron(x.shape[1], y.shape[1]).to(device)

model

MultiLayerPerceptron(
  (hidden): Linear(in_features=1080, out_features=500, bias=True)
  (output): Linear(in_features=500, out_features=27, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

def train_fn(model, optimizer, loss_fn, device):
    ave_loss = 0
    count = 0
    
    for i, data in enumerate(x):
        data = x[i]
        targets = y[i]
        
        # Forward
        predictions = model.forward(data)
        
        predictions = F.softmax(predictions, dim=-1)
        
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        count += 1
        ave_loss += loss.item()
    
    ave_loss = ave_loss / count

    return ave_loss

epochs = 1000

average_losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(model, optimizer, criterion, device)
    
    average_losses.append(ave_loss)
        
    print("Ave Loss: {}".format(ave_loss))
    
    generated_sentence = callback(model)
    
    print("Generated sentence:")
    print(generated_sentence)
    print("Length: {}".format(len(generated_sentence)))

Epoch: 0
Ave Loss: 3.2957752937280036
Generated sentence:
eqefud,rssmwlbha
rijq mafvm orjjlefjrjge
Length: 40
Epoch: 1
Ave Loss: 3.295512867434589
Generated sentence:
viip
rg
yklkuadmm,fc odhi oiyyvmqnnsepaa
Length: 40
Epoch: 2
Ave Loss: 3.295240372275385
Generated sentence:
ifdbatlojweifwnjwnnqi othkqcwt
aljvvb yv
Length: 40
Epoch: 3
Ave Loss: 3.2949562971142754
Generated sentence:
lirkhvvghq
bupiwm,bwsljatvsawvciyrrttj
 
Length: 40
Epoch: 4
Ave Loss: 3.2946593346803086
Generated sentence:

l,vlv wbldoy,jrqeolijmyphnamvpjt, rshyu
Length: 40
Epoch: 5
Ave Loss: 3.2943467921104985
Generated sentence:
a, oykbtjocppbtlrniwdbqohell amontbsypde
Length: 40
Epoch: 6
Ave Loss: 3.2940162840672738
Generated sentence:
o gb meijubmdu
brawilskt,myht
mhag lbkpq
Length: 40
Epoch: 7
Ave Loss: 3.2936657166135483
Generated sentence:
ptn,hohcpnfdtahvrrhvdcj hfot,rigftc,tysr
Length: 40
Epoch: 8
Ave Loss: 3.2932924855734416
Generated sentence:
cryyr 
ygmnayd sikhh,jkykfjekwwabfpiesab
Length: 40
Epoch: 9
Ave

Ave Loss: 3.2647336093699875
Generated sentence:
hkat hls e
 s, i  anlelterahowe twlsgsha
Length: 40
Epoch: 76
Ave Loss: 3.264314271401668
Generated sentence:
 e etiihhhenhoach omgtehltoaiehde h saoa
Length: 40
Epoch: 77
Ave Loss: 3.263893797777701
Generated sentence:
yeoamerrlaoc aor hatorioalnn
tharrsseae,
Length: 40
Epoch: 78
Ave Loss: 3.263473739946522
Generated sentence:
 en,si s sr hhfihrt odthas n rt dsrdehr 
Length: 40
Epoch: 79
Ave Loss: 3.263052640905703
Generated sentence:
ehieoiar f ear   sar
o et lls
eenl tea s
Length: 40
Epoch: 80
Ave Loss: 3.2626325174230306
Generated sentence:
aeeeonainig c hfnsstnlnh nolltrss nheisd
Length: 40
Epoch: 81
Ave Loss: 3.262213033178578
Generated sentence:
 rhh   hha ehed hlnsthhtaat   snstcoh
is
Length: 40
Epoch: 82
Ave Loss: 3.2617939186556906
Generated sentence:
htcaee ll n  hindciln coaachn   srhn rbh
Length: 40
Epoch: 83
Ave Loss: 3.2613764534825864
Generated sentence:
tse oheactwk 
 h ol
, ehm 
ehola hehhee 
Length: 40
Epoch: 84
Ave Lo

Ave Loss: 3.242478018221648
Generated sentence:
ineto noh eoor  e  aa ee sa ime enai las
Length: 40
Epoch: 150
Ave Loss: 3.2423450013865596
Generated sentence:
e   eraeoee  a ee e    f  i o e h   e 
e
Length: 40
Epoch: 151
Ave Loss: 3.242215749722172
Generated sentence:
 taae e d ie n e e a  s t e hhh  hn  h f
Length: 40
Epoch: 152
Ave Loss: 3.242090037479493
Generated sentence:
e  asa h  eae h o e  he fa nhe 
ta e h s
Length: 40
Epoch: 153
Ave Loss: 3.241967725293072
Generated sentence:
  oes ht h 
oh
ho  h  eihmarc ee aihha  
Length: 40
Epoch: 154
Ave Loss: 3.24184905388505
Generated sentence:
he  os o  e oed fa  aae  th  a s  ta ihn
Length: 40
Epoch: 155
Ave Loss: 3.2417334945881424
Generated sentence:
hetaneoihh net  he  thi aissoto
a h     
Length: 40
Epoch: 156
Ave Loss: 3.2416211188127453
Generated sentence:
naa eie  ee eh   see  aee hr  eh a hg eo
Length: 40
Epoch: 157
Ave Loss: 3.2415121327275815
Generated sentence:
haeenhn e hent   as  ieefesi   t  f  e  
Length: 40
Epoch: 15

Ave Loss: 3.238552733895859
Generated sentence:
a  ie eeh  acs soa h aaoe toi   he h s  
Length: 40
Epoch: 224
Ave Loss: 3.238541809257102
Generated sentence:
 ael re roe e frh  an ee e  eaf e toe s 
Length: 40
Epoch: 225
Ave Loss: 3.2385313269020854
Generated sentence:
  e t i  ahe ah  ae aae h i h son  n ee 
Length: 40
Epoch: 226
Ave Loss: 3.2385212361524647
Generated sentence:
 he  ae aoh  aoie he  ani  h   aene h  a
Length: 40
Epoch: 227
Ave Loss: 3.2385115347046782
Generated sentence:
e  he  aes ahs thl ae  na s th  ahe eh s
Length: 40
Epoch: 228
Ave Loss: 3.2385022029784567
Generated sentence:
e  ia ee he he hae se he hed  h ahe a oe
Length: 40
Epoch: 229
Ave Loss: 3.238493245580922
Generated sentence:
ene h    a oees h s tae  he ie e e s he 
Length: 40
Epoch: 230
Ave Loss: 3.2384846360211212
Generated sentence:
 haes hn  ha se n  thnhe eahis ooe ahs  
Length: 40
Epoch: 231
Ave Loss: 3.2384763696919316
Generated sentence:
e s h  ih  he  ac sse  a he  an e hsohe 
Length: 40
Epoch:

Ave Loss: 3.2382952441339907
Generated sentence:
 he  rhs aer aio  in  ehs  hl ee fee s h
Length: 40
Epoch: 298
Ave Loss: 3.2382947949395664
Generated sentence:
s srdeihe  aa e ohe hade a  hh eohe ora 
Length: 40
Epoch: 299
Ave Loss: 3.23829438145034
Generated sentence:
 he  sc tnee ha   he ke ors  aoh noa s o
Length: 40
Epoch: 300
Ave Loss: 3.238293967961113
Generated sentence:
ie  aa  eoe stee  h  ha inh se hn e sran
Length: 40
Epoch: 301
Ave Loss: 3.2382935740521566
Generated sentence:
aoee  he se   ahasa re ees  te  he tse 

Length: 40
Epoch: 302
Ave Loss: 3.238293217000178
Generated sentence:
 ha aa tae  h ae ths  hnsoe o a  aeee he
Length: 40
Epoch: 303
Ave Loss: 3.2382928668588833
Generated sentence:
 ae h  ae  hrf ahanh a  le  te   oic ea 
Length: 40
Epoch: 304
Ave Loss: 3.2382925339942967
Generated sentence:
  in  e  me e he le hs haseornm  is tae 
Length: 40
Epoch: 305
Ave Loss: 3.238292218406419
Generated sentence:
 
eiotsa  ne are  ae  ihh s hlsee ha eee
Length: 40
Epoch: 3

...

Ave Loss: 3.2382855484451074
Generated sentence:
 he  te hescs loekahr eo eo  me ei tls a
Length: 40
Epoch: 668
Ave Loss: 3.2382855484451074
Generated sentence:
e ha sh s tae aof oe oe s  io  oe se oe 
Length: 40
Epoch: 669
Ave Loss: 3.2382855484451074
Generated sentence:
 eis oer nae  he   on  ho es lee mnt  he
Length: 40
Epoch: 670
Ave Loss: 3.2382855472933265
Generated sentence:
 he ahe safe is  hin tes ne trah  ha iah
Length: 40
Epoch: 671
Ave Loss: 3.2382855472933265
Generated sentence:
tois ohnl 
ane  h he ioe onl st hek nr a
Length: 40
Epoch: 672
Ave Loss: 3.2382855472933265
Generated sentence:
a ne  h he moreoolss aee  nr  e me e ha 
Length: 40
Epoch: 673
Ave Loss: 3.2382855472933265
Generated sentence:
ohe  ae  an  aan  ah  aa te s aiselhe  h
Length: 40
Epoch: 674
Ave Loss: 3.2382855472933265
Generated sentence:
e ln a a re e she taef hsfeoonoi h ie  h
Length: 40
Epoch: 675
Ave Loss: 3.2382855472933265
Generated sentence:
ore oin  aeoe o ri  ane  a s fereeanl ta
Length: 40
Epo

Ave Loss: 3.238285543837985
Generated sentence:
 aacens at ae  ee 
el s ia ae a s ae  ra
Length: 40
Epoch: 742
Ave Loss: 3.238285543837985
Generated sentence:
rs tees a a seroie  ra  aee hte has aeh 
Length: 40
Epoch: 743
Ave Loss: 3.238285543837985
Generated sentence:
e mirn ie  ee he  r or oc nae ne he  aes
Length: 40
Epoch: 744
Ave Loss: 3.238285543837985
Generated sentence:
he  oees aeie nr in te he ah ota helsaml
Length: 40
Epoch: 745
Ave Loss: 3.238285543837985
Generated sentence:
 ais edenaa h s o sel he sio to hefie ta
Length: 40
Epoch: 746
Ave Loss: 3.238285543837985
Generated sentence:
  ores hi  anraes tn s ees lil seeadeiai
Length: 40
Epoch: 747
Ave Loss: 3.238285543837985
Generated sentence:
seoain ais t  h he meos ahiscornoe s ta 
Length: 40
Epoch: 748
Ave Loss: 3.238285543837985
Generated sentence:
at  the sfthe hn atsecoon
 te sa fie  ah
Length: 40
Epoch: 749
Ave Loss: 3.238285543837985
Generated sentence:
 t oe fer  a   aae oa s aes ane hr sreef
Length: 40
Epoch: 750
A

Ave Loss: 3.238285541534424
Generated sentence:
 mt ee  na te er he t ho eiha teosee cam
Length: 40
Epoch: 816
Ave Loss: 3.238285541534424
Generated sentence:
a geras he  hane ha aederefeols  roeo hi
Length: 40
Epoch: 817
Ave Loss: 3.238285541534424
Generated sentence:
cosh  hel san e
y ofsenoae   he aesera s
Length: 40
Epoch: 818
Ave Loss: 3.238285541534424
Generated sentence:
e h mie he  aasee reaernna r  n  aaee nn
Length: 40
Epoch: 819
Ave Loss: 3.238285541534424
Generated sentence:
 ao
e  an  or he e els his tanaess ohe  
Length: 40
Epoch: 820
Ave Loss: 3.238285541534424
Generated sentence:
 oid  hi  hos ahi iee he  ahe iahs ih  i
Length: 40
Epoch: 821
Ave Loss: 3.238285541534424
Generated sentence:
 ano  h saew
n
  a ee the  ilielhh a soe
Length: 40
Epoch: 822
Ave Loss: 3.238285541534424
Generated sentence:
e hhe  an eel mh  ne eolm h se aos co ie
Length: 40
Epoch: 823
Ave Loss: 3.238285541534424
Generated sentence:
e hh  ia oeehm a d ee  ha aesnee hnf  oe
Length: 40
Epoch: 824
A

Ave Loss: 3.238285541534424
Generated sentence:
o fann  ha ce ne e his tolis nae eis  h 
Length: 40
Epoch: 890
Ave Loss: 3.238285541534424
Generated sentence:
 noeey fa  tase ohae  he h his nores re 
Length: 40
Epoch: 891
Ave Loss: 3.238285541534424
Generated sentence:
e  hife hna de ae w s   aeel an  is to i
Length: 40
Epoch: 892
Ave Loss: 3.238285541534424
Generated sentence:
 en haa see ahi  a eaehs aas aer  a s ae
Length: 40
Epoch: 893
Ave Loss: 3.238285541534424
Generated sentence:
 tasee har  ho soa se aoadc fmee ra i se
Length: 40
Epoch: 894
Ave Loss: 3.238285541534424
Generated sentence:
n  she a tke sntaes he wresotl  rae oe  
Length: 40
Epoch: 895
Ave Loss: 3.238285541534424
Generated sentence:
 ha te  ahe sraereew aas tes  oh saecosh
Length: 40
Epoch: 896
Ave Loss: 3.238285541534424
Generated sentence:
ns an erof, hnof aeoec  haeshls  aisaes 
Length: 40
Epoch: 897
Ave Loss: 3.238285541534424
Generated sentence:
 eo hohiseaoena  a see ,sr ol see caa oe
Length: 40
Epoch: 898
A

Ave Loss: 3.238285541534424
Generated sentence:
e,  nae  hh  o,nsane sh  hoheradegtstaah
Length: 40
Epoch: 964
Ave Loss: 3.238285541534424
Generated sentence:
the ath he
 n he the ohns aos o 
 ca
 te
Length: 40
Epoch: 965
Ave Loss: 3.238285541534424
Generated sentence:
hed a ne eh ha lera oe a taes na  a e  i
Length: 40
Epoch: 966
Ave Loss: 3.238285541534424
Generated sentence:
mr e h ae aesscsfeefnr  a me sfe aafis  
Length: 40
Epoch: 967
Ave Loss: 3.238285541534424
Generated sentence:
ia he ane sa se sr les  earaeoh s aahe k
Length: 40
Epoch: 968
Ave Loss: 3.238285541534424
Generated sentence:
hi
 thn aes ho msee
o a s en ehan i r ee
Length: 40
Epoch: 969
Ave Loss: 3.238285541534424
Generated sentence:
 nr ch s a oeif tre hn
a  or shw s  tae 
Length: 40
Epoch: 970
Ave Loss: 3.238285541534424
Generated sentence:
 aie  he arserels ta  ae  io  heaee h  h
Length: 40
Epoch: 971
Ave Loss: 3.238285541534424
Generated sentence:
if hifre ha ahe or  oris ha  ha aie ane 
Length: 40
Epoch: 972
A